In [1]:
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import models_v2
from tqdm import tqdm

C:\Users\kamil\anaconda3\envs\vit_sampling\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from patch_sampler import *
from torchvision.datasets import Flowers102

from src.color_histograms_entropy_sampler import ColorHistogramsEntropySampler
from src.edge_detector_limited_points_sampler import EdgeDetectorLimitedPointsSampler

ModuleNotFoundError: No module named 'utils'

In [3]:
class CustomDataset(Dataset):
    def __init__(self, base_dataset, split, sampler):
        self.base_dataset = base_dataset
        self.sampler = sampler
        self.split = split
        self.augmentations = self._get_augmentations()
    
    def _get_augmentations(self):
        train_transforms = [
            transforms.Resize((240, 240)),
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(degrees=(-45, 45)),
            transforms.RandomApply([transforms.GaussianBlur(kernel_size=5)], p=0.5),
            transforms.Normalize(mean=[0.4330, 0.3819, 0.2964], std=[0.2621, 0.2133, 0.2248])
        ]
        test_transforms = [
            transforms.Resize((240, 240)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.4330, 0.3819, 0.2964], std=[0.2621, 0.2133, 0.2248])
        ]

        if self.split == 'train':
            return transforms.Compose(train_transforms)
        else:
            return transforms.Compose(test_transforms)

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        image, label = self.base_dataset[idx]
        image = self.augmentations(image)
        x, coords = self.sampler(image)
        return x, coords, label, image

CustomDataset jest klasą przetwarzającą dataset np. Flowers102 na patche i koordynaty w formacie przyjmowanym przez ElsaticVit.
Tzn. w momencie wywołania metody getitem(index) z oryginalnego datasetu pobierany jest obraz o danym indeksie, nakładane są na niego augmentacje (w zależności od tego, czy jest to zbiór treningowy czy nie - dane argumentem 'split'), a następnie obraz dzielony jest na patche zgodnie z metodą daną argumentem 'sampler'.
Normalizacja zbiorów powinna być zależna od średniej i odchylenia standardowego dla danego datasetu.

Dataset ten jest dostosowany do możliwości wykorzystania metody uczenia przez destylację, dlatego zwraca również obraz jako całość w postaci argumentu 'image'. Z powodu braku wykorzystania destylacji w aktualnym sposobie treningu, element ten możnaby pominąć, jednak należałoby wtedy wziąć to pod uwagę również w pętli treningowej

Poniżej, przykłady użycia - GridSamplerV2 z domyślną liczbą patchy (14,14) dzieli obraz na standardowy grid. W ramach przykładu wykorzystano dataset Flowers102.
W miejsce GridSamplerV2 należy podstawić dowolną opracowaną przez nas metodę podziału na patche, tak długo jak patche mają rozmiar 16x16, a obraz początkowy ma rozmiar 224x224. Koordynaty muszą mieścić się w tym zakresie i być zgodne z opisanymi w pracy, a więc przentować lewy górny i prawy dolny róg

In [ ]:
#sampler = GridSamplerV2(patches_num_yx=(14,14))
sampler =
base_train_dataset = Flowers102('Flowers102', split='train',download=True)
base_val_dataset = Flowers102('Flowers102', split='val',download=True)
flowers_train_dataset = CustomDataset(base_train_dataset, 'train', sampler)
flowers_valid_dataset = CustomDataset(base_val_dataset, 'val', sampler)

Standardowe dataloadery dla zbioru treningowego i walidacyjnego

In [ ]:
batch_size = 16  # Set an appropriate batch size
train_loader = DataLoader(flowers_train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(flowers_valid_dataset, batch_size=batch_size, shuffle=False)

Wczytujemy model oparty na standardowym modelu deit jednak przyjmującym patche i koordynaty zamiast zdjęć. Wczytujemy również zapamiętany state uczenia zawierający przede wszystkim wagi wytrenowanego modelu, znajdujące się w słowniku pod nazwą model (dla mnie najlepiej zadziałał chceckpoint 'elastic-224-70random30grid.pth'). W systemie Windows konieczne może być przetworzenie ścieżki do pliku jak niżej, w przeciwnym wypadku powoduje błąd wykonania

In [ ]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [ ]:
model = models_v2.deit_base_patch16_LS()
state = torch.load('elastic-224-70random30grid.pth')
loaded_model = state['model']

Liczba klas w pretrenowanym modelu (1000) najprawdopodobniej nie zgadza się z liczbą klas w użytym datasecie. W przykładowym przypadku liczba klas wynosi 102, więc trzeba zresetować ostatnią warstwę - klasyfikator. Model posiada taką metodę wbudowaną, przyjmujacą liczbę klas jako parametr

In [ ]:
model.reset_classifier(num_classes=102) #for flowers

Przenosimy model na GPU (CUDA) jeśli jest taka możliwość

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Fine tunowanie modelu powinno wykorzystać pretrenowane wagi. W tym celu zalecam rozpoczęcie treningu od trenowania wyłącznie nowopowstałej warstwy klasyfikacyjnej (head). W tym celu ustawiamy liczenie gradientów wyłącznie dla tej warstwy, wyłączając je dla pozostałych warstw

In [ ]:
for p in model.parameters():
    p.requires_grad = False
model.head.weight.requires_grad = True
model.head.bias.requires_grad = True

Ponieważ warstwa klasyfikacyjna jest pusta, wymaga większej liczby epok uczenia, jak również może korzystać z wyższego learning rate'u

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.1)
epochs = 20

Standardowa pętla uczenia wraz z walidacją po każdej epoce. Wykorzystano tqdm w celu lepszej prezentacji postępu uczenia

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    train_correct = 0
    train_outputs = 0
    for i, data in enumerate(tqdm(train_loader), 0):
        x, coords, labels, images = data
        x, coords, labels, images = x.to(device), coords.to(device), labels.to(device), images.to(device)
        optimizer.zero_grad()
        outputs = model(x, coords)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_correct += (torch.argmax(outputs, dim=-1) == labels).sum().item()
        train_outputs += outputs.shape[0]

        running_loss += loss
    model.eval()
    total_correct = 0
    total_outputs = 0
    with torch.no_grad():
        for i, data in enumerate(tqdm(valid_loader), 0):
            x, coords, labels,_ = data
            x, coords, labels = x.to(device), coords.to(device), labels.to(device)
            outputs = model(x, coords)
            correct = (torch.argmax(outputs, dim=-1) == labels).sum().item()
            total_correct += correct
            total_outputs += outputs.shape[0]
    print(f"[Epoch {epoch + 1}] Loss: {running_loss / i:.3f}, Train Acc: {train_correct/train_outputs:.3f}, Valid Acc: {total_correct/total_outputs:.3f}")



Poprawiony model można zapisać

In [ ]:
torch.save(model.state_dict(), "85-12.pth")

A następnie wczytać do dalszych testów po wcześniejszym zdefiniowaniu tak samo jak wcześniej

In [ ]:
model.load_state_dict(torch.load("85-12.pth"))

W celu poprawienia accuracy, zalecam dodatkowo dotrenować warstwy powiązane z patchami i pozycjami, ale dopiero po wytrenowaniu klasyfikatora

In [ ]:
for p in model.parameters():
    p.requires_grad = False
model.head.weight.requires_grad = True
model.head.bias.requires_grad = True
model.pos_embed.requires_grad = True
model.patch_embed.proj.weight.requires_grad = True
model.patch_embed.proj.bias.requires_grad = True
for p in model.patch_embed.parameters():
    p.requires_grad = True

Mając dotrenowane te parametry można ponowić trening odblokowując warstwy atencji:

In [ ]:
for name_p, p in model.named_parameters():
    if '.attn.' in name_p:
        p.requires_grad = True
    else:
        p.requires_grad = False
model.head.weight.requires_grad = True
model.head.bias.requires_grad = True
model.pos_embed.requires_grad = True
model.patch_embed.proj.weight.requires_grad = True
model.patch_embed.proj.bias.requires_grad = True
for p in model.patch_embed.parameters():
    p.requires_grad = True

Stosując te trzy stopnie fine tuningu udało mi się uzyskać accuracy na zbiorze walidacyjnym na poziomie 85% i ponad 84% na zbiorze testowym

Dodatkowo model ma pewne parametry, między innymi trzy rodzaje dropoutu, które można ustawić w celu zmniejszenia overfittingu. Wtedy część wag, atencji lub patchy jest losow ignorowana podczas uczenia. Trzeba z tym jednak uważać, szczególnie jeśli chodzi o drop_path_rate

In [ ]:
model = models_v2.deit_base_patch16_LS(drop_rate=0.25, attn_drop_rate=0.25, drop_path_rate=0.0)

Deit jest szczególnym przypadkiem ViT, który umożliwia trenowanie z użyciem destylacji. Oznacza to, że można wykorzystać inną wytrenowaną sieć. Pozwala to wykorzystać nie tylko klasyfikację, ale również poszczególne prawdopodobieństwa zwracane przez sieć, co w pewnych sytuacjach może ułatwiać uczenie. W przypadku pretrenowanego Elastic-ViT dla datasetu Flowers102 nie zauważyłem szczególnej poprawy dokładności, a konieczność zarówno wytrenowania drugiego modelu, jak również konieczność ewaluacji wyników na dwóch modelach znacząco wydłuża czas treningu. Na ten moment nie widzę sensu zaciemniania obrazu poprzez dodanie tego kroku.